# 画像を使って音楽生成をしてみよう

- ライブラリのインポート
- model_base_capfilt_large.pthのダウンロード
- 画像からテキストへの変換
- テキストから音楽への変換

## 1.ライブラリのインポート

In [1]:
!pip install timm
!pip install fairscale
!pip install ftfy regex tqdm
!pip install transformers -U
!pip install ftfy regex tqdm transformers==4.21.3 timm==0.4.12 fairscale==0.4.4
!git clone https://github.com/salesforce/BLIP
!pip install -U sentence-transformers
!pip install httpx
!pip install wget


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached transformers-4.27.4-py3-none-any.whl (6.8 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.21.3
    Uninstalling transformers-4.21.3:
      Successfully uninstalled transformers-4.21.3



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached transformers-4.21.3-py3-none-any.whl (4.7 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.27.4
    Uninstalling transformers-4.27.4:
      Successfully uninstalled transformers-4.27.4



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
fatal: destination path 'BLIP' already exists and is not an empty directory.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%cd ./BLIP
import gc
import numpy as np
import os
import pandas as pd
import requests
import torch
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from IPython.display import Audio, display
from PIL import Image
from torch import nn
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
from models.blip import blip_decoder
from sentence_transformers import SentenceTransformer
import httpx
import json
import wget

C:\Users\student\Desktop\img2music\BLIP


OSError: [WinError 127] 指定されたプロシージャが見つかりません。 Error loading "C:\ProgramData\Anaconda3\lib\site-packages\torch\lib\torch_cuda.dll" or one of its dependencies.

## 2.model_base_capfilt_large.pthのダウンロード
model_base_capfilt_large.pthというモデルのファイルをwgetを用いてfile_nameに格納します.

In [ ]:
site_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_capfilt_large.pth'
file_name = wget.download(site_url)
print(file_name)

## 3.画像からテキストの変換

画像から画像を説明するテキストを変換します.
まずは画像を選択します.

今回はhttps://cdn.pixabay.com/photo/2017/12/11/15/34/lion-3012515_960_720.jpg　の画像で行います。

In [ ]:
img_url = 'https://cdn.pixabay.com/photo/2017/12/11/15/34/lion-3012515_960_720.jpg'
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
display(raw_image)

次にこの画像をテキストに変換します.

先ほどダウンロードしたモデルを用いて,テキストを生成します.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 
def load_demo_image(image_size, device):

 
    w, h = raw_image.size
 
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size), interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
    ])
    image = transform(raw_image).unsqueeze(0).to(device)
    return image
 
 
if __name__ == '__main__':
 
    image_size = 384
    image = load_demo_image(image_size=image_size, device=device)
    model_path = file_name
    model = blip_decoder(pretrained=model_path, image_size=image_size, vit='base')
    model.eval()
    model = model.to(device)
 
    with torch.no_grad():
        caption = model.generate(image, sample=False, num_beams=3, max_length=20, min_length=5)
        print('テキスト変換: ' + caption[0])

このような結果になりました.

In [ ]:
%cd ..

In [ ]:
# パッケージのインストール
!git clone https://github.com/haoheliu/AudioLDM.git
%cd AudioLDM
!pip install -e . --user

In [ ]:
!mkdir ckpt
!wget https://zenodo.org/record/7600541/files/audioldm-s-full?download=1 -O ckpt/audioldm-s-full.ckpt

In [ ]:
# 推論の実行
!python scripts/text2sound.py -t "2 cats are fighting" 

## 4.テキストから音楽への変換

先ほど変換したテキストを用いて音楽を生成していきます.
Sentence Transfomerを用いてテキストを文章ベクトルに変換し,[mubert](https://mubert.com/)が提供している音楽生成をAPIに送ることで音楽を生成します.

emailに自分のメールアドレスを入力して実行し, 自分だけのtokenを各自で取得してください.

In [ ]:
minilm = SentenceTransformer('all-MiniLM-L6-v2')
mubert_tags_string = 'tribal,action,kids,neo-classic,run 130,pumped,jazz / funk,ethnic,dubtechno,reggae,acid jazz,liquidfunk,funk,witch house,tech house,underground,artists,mystical,disco,sensorium,r&b,agender,psychedelic trance / psytrance,peaceful,run 140,piano,run 160,setting,meditation,christmas,ambient,horror,cinematic,electro house,idm,bass,minimal,underscore,drums,glitchy,beautiful,technology,tribal house,country pop,jazz & funk,documentary,space,classical,valentines,chillstep,experimental,trap,new jack swing,drama,post-rock,tense,corporate,neutral,happy,analog,funky,spiritual,sberzvuk special,chill hop,dramatic,catchy,holidays,fitness 90,optimistic,orchestra,acid techno,energizing,romantic,minimal house,breaks,hyper pop,warm up,dreamy,dark,urban,microfunk,dub,nu disco,vogue,keys,hardcore,aggressive,indie,electro funk,beauty,relaxing,trance,pop,hiphop,soft,acoustic,chillrave / ethno-house,deep techno,angry,dance,fun,dubstep,tropical,latin pop,heroic,world music,inspirational,uplifting,atmosphere,art,epic,advertising,chillout,scary,spooky,slow ballad,saxophone,summer,erotic,jazzy,energy 100,kara mar,xmas,atmospheric,indie pop,hip-hop,yoga,reggaeton,lounge,travel,running,folk,chillrave & ethno-house,detective,darkambient,chill,fantasy,minimal techno,special,night,tropical house,downtempo,lullaby,meditative,upbeat,glitch hop,fitness,neurofunk,sexual,indie rock,future pop,jazz,cyberpunk,melancholic,happy hardcore,family / kids,synths,electric guitar,comedy,psychedelic trance & psytrance,edm,psychedelic rock,calm,zen,bells,podcast,melodic house,ethnic percussion,nature,heavy,bassline,indie dance,techno,drumnbass,synth pop,vaporwave,sad,8-bit,chillgressive,deep,orchestral,futuristic,hardtechno,nostalgic,big room,sci-fi,tutorial,joyful,pads,minimal 170,drill,ethnic 108,amusing,sleepy ambient,psychill,italo disco,lofi,house,acoustic guitar,bassline house,rock,k-pop,synthwave,deep house,electronica,gabber,nightlife,sport & fitness,road trip,celebration,electro,disco house,electronic'
mubert_tags = np.array(mubert_tags_string.split(','))
mubert_tags_embeddings = minilm.encode(mubert_tags)

def get_track_by_tags(tags, pat, duration, maxit=20, autoplay=False, loop=False):
  if loop:
    mode = "loop"
  else:
    mode = "track"
  r = httpx.post('https://api-b2b.mubert.com/v2/RecordTrackTTM', 
      json={
          "method":"RecordTrackTTM",
          "params": {
              "pat": pat, 
              "duration": duration,
              "tags": tags,
              "mode": mode
          }
      })

  rdata = json.loads(r.text)
  assert rdata['status'] == 1, rdata['error']['text']
  trackurl = rdata['data']['tasks'][0]['download_link']

  print('Generating track ', end='')
  for i in range(maxit):
      r = httpx.get(trackurl)
      if r.status_code == 200:
          display(Audio(trackurl, autoplay=autoplay))
          break
      time.sleep(1)
      print('.', end='')

def find_similar(em, embeddings, method='cosine'):
    scores = []
    for ref in embeddings:
        if method == 'cosine': 
            scores.append(1 - np.dot(ref, em)/(np.linalg.norm(ref)*np.linalg.norm(em)))
        if method == 'norm': 
            scores.append(np.linalg.norm(ref - em))
    return np.array(scores), np.argsort(scores)

def get_tags_for_prompts(prompts, top_n=3, debug=False):
    prompts_embeddings = minilm.encode(prompts)
    ret = []
    for i, pe in enumerate(prompts_embeddings):
        scores, idxs = find_similar(pe, mubert_tags_embeddings)
        top_tags = mubert_tags[idxs[:top_n]]
        top_prob = 1 - scores[idxs[:top_n]]
        if debug:
            print(f"Prompt: {prompts[i]}\nTags: {', '.join(top_tags)}\nScores: {top_prob}\n\n\n")
        ret.append((prompts[i], list(top_tags)))
    return ret


# emailに自分のメールアドレスを記入する
email = "*******************@gmail.com" 

r = httpx.post('https://api-b2b.mubert.com/v2/GetServiceAccess', 
    json={
        "method":"GetServiceAccess",
        "params": {
            "email": email,
            "license":"ttmmubertlicense#f0acYBenRcfeFpNT4wpYGaTQIyDI4mJGv5MfIhBFz97NXDwDNFHmMRsBSzmGsJwbTpP1A6i07AXcIeAHo5",
            "token":"4951f6428e83172a4f39de05d5b3ab10d58560b8",
            "mode": "loop"
        }
    })

rdata = json.loads(r.text)
assert rdata['status'] == 1, "probably incorrect e-mail"
pat = rdata['data']['pat']

#tokenが表示される
#print(f'Got token: {pat}')

では先ほどのテキストを用いて音楽生成を行います.

durationで音楽の長さ,loopで曲をループするかを決めることができます.

In [ ]:
import time

print('text = ', caption[0])
prompt = caption[0]
duration = 30 
loop = False 

def generate_track_by_prompt(prompt, duration, loop=False):
  _, tags = get_tags_for_prompts([prompt,])[0]
  print('tags = ', tags)
  
  try:
    get_track_by_tags(tags, pat, duration, autoplay=True, loop=loop)
  except Exception as e:
    print(str(e))
  print('\n')

generate_track_by_prompt(prompt, duration, loop)